In [ ]:
import cv2
import numpy as np
from os import listdir
from google.colab.patches import cv2_imshow

def matching(testList):
  for filename in listdir('/content/drive/My Drive/Final Year Project/Sample Images'):
    SampImg = cv2.imread("/content/drive/My Drive/Final Year Project/Sample Images/"+filename, cv2.IMREAD_GRAYSCALE)
    # ORB Detector
    orb = cv2.ORB_create()
    kp2, des2 = orb.detectAndCompute(SampImg, None)
    totMatch = 0
    for i in range(len(testList)):
      kp1, des1 = orb.detectAndCompute(testList[i], None)
      # Brute Force Matching
      bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
      matches = bf.match(des1, des2)
      matches = sorted(matches, key = lambda x:x.distance)
      matches = [m for m in matches if m.distance<60]
      #print(len(matches))
      totMatch += len(matches)
      matching_result = cv2.drawMatches(testList[i], kp1, SampImg, kp2, matches, None, flags=2)
      #cv2_imshow(testList[i])
      #cv2_imshow(SampImg)
      #cv2_imshow(matching_result)
    print(filename+" Total = ",totMatch)

# Read image
for filename in listdir('/content/drive/My Drive/Final Year Project/Test Images'):
  testImg = cv2.imread("/content/drive/My Drive/Final Year Project/Test Images/"+filename, cv2.IMREAD_GRAYSCALE)
  print("\n"+filename)
  #cv2_imshow(testImg) cv2.cvtColor(testImg, cv2.COLOR_BGR2GRAY)
  cp = testImg
  th,im = cv2.threshold(cp, 127, 255, cv2.THRESH_BINARY_INV)
  # Setup SimpleBlobDetector parameters.
  params = cv2.SimpleBlobDetector_Params()
  # Change thresholds
  params.minThreshold = 10
  params.maxThreshold = 200
  # Filter by Area.
  params.filterByArea = True
  params.minArea = 1000
  # Filter by Color
  params.filterByColor = True
  params.blobColor=0

  # Filter by Circularity
  params.filterByCircularity = True
  params.minCircularity = 0.4
  # Filter by Convexity
  params.filterByConvexity = True
  params.minConvexity = .15

  # Filter by Inertia
  params.filterByInertia = True
  params.minInertiaRatio = 0.01
  # Create a detector with the parameters
  #detector = cv2.SimpleBlobDetector(params)
  detector = cv2.SimpleBlobDetector_create(params)
  # Detect blobs.
  keypoints = detector.detect(im)
  # Draw detected blobs as red circles.
  # cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures
  # the size of the circle corresponds to the size of blob
  im_with_keypoints = cv2.drawKeypoints(im, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
  # Show blobs
  edged = cv2.Canny(im, 30, 200)
  contours, hierarchy = cv2.findContours(edged,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
  if len(contours) < 100:
    cv2.drawContours(cp, contours, -1, (0,0,255), 2)
    #cv2_imshow(cp)
    ROIList = list()
    for cnt in contours:
      #TestImg = [[0]*im.shape[1]]*im.shape[0]
      x,y,w,h = cv2.boundingRect(cnt)
      if w>5 and h>5:
        y1 = y-30 if y-30>0 else 0
        y2 = y+h+30 if y+h+30<testImg.shape[0] else testImg.shape[0]
        x1 = x-30 if x-30>0 else 0
        x2 = x+h+30 if x+h+30<testImg.shape[1] else testImg.shape[1]
        ROI = testImg[y1:y2, x1:x2]
        ROIList.append(ROI)
        #cv2_imshow(ROI)
    matching(ROIList)